In [1]:
import pandas as pd
import numpy as np
import problem

## Utils

In [2]:
### LAG ###

lag = True

if lag==True:
    date_key = 'Purchase Date'
elif lag==False:
    date_key = 'DateOfDeparture'

### LAG ###

In [3]:
def purchase_date(df):
    df = df.copy()
    df_data1 = pd.read_csv(os.path.join('data', 'train.csv.bz2'))
    df_data2 = pd.read_csv(os.path.join('data', 'test.csv.bz2'))
    df_data = pd.concat([df_data1, df_data2])
    df_data['DateOfDeparture'] = pd.to_datetime(df_data['DateOfDeparture'])
    # df_data['avg_wtd_dep'] = df_data['WeeksToDeparture']
    #df_data = df_data.rename(columns={'WeeksToDeparture':'avg_wtd_arr'})

    #df_data["Purchase Date"] = pd.to_datetime("1970-01-01") + pd.DateOffset(days=time_diff)
    
    df_data = df_data.drop(columns=['std_wtd', 'log_PAX'])
    print(df_data.columns)

    df_dep = df_data.copy().drop(columns=['Arrival']).groupby(['DateOfDeparture', 'Departure']).mean().reset_index()
    df_dep = df_dep.rename(columns={'WeeksToDeparture':'WTD Dep'})
    df_arr = df_data.copy().drop(columns=['Departure']).groupby(['DateOfDeparture', 'Arrival']).mean().reset_index()
    df_dep = df_dep.rename(columns={'WeeksToDeparture':'WTD Arr'})


    # time_diff = (df_dep['DateOfDeparture'] - pd.to_datetime("1970-01-01") ).dt.days - 7 * df_dep['WeeksToDeparture']
    # df_dep['avg purchase date dep'] = pd.to_datetime("1970-01-01") + time_diff.apply(np.ceil).apply(lambda x: pd.Timedelta(x,               unit='D'))
    # df_dep = df_dep.drop(columns=['WeeksToDeparture'])
    df = df.merge(df_arr, how='left', on=['DateOfDeparture', 'Arrival'])
    df = df.merge(df_dep, how='left', on=['DateOfDeparture', 'Departure'])

    df['WTD Arr'].fillna(df['WTD Dep'], inplace=True)
    del df['WTD Dep']

    df['WTD Arr'] = df.fillna(df['WTD Arr'].mean())

    time_diff = (df_arr['DateOfDeparture'] - pd.to_datetime("1970-01-01") ).dt.days - 7 * df_arr['WeeksToDeparture']
    df_arr['avg purchase date'] = pd.to_datetime("1970-01-01") + time_diff.apply(np.ceil).apply(lambda x: pd.Timedelta(x, unit='D'))
    df_arr = df_arr.drop(columns=['WeeksToDeparture'])

    df['p_year'] = df['avg purchase date'].dt.year
    df['p_month'] = df['avg purchase date'].dt.month
    df['p_week'] = df['avg purchase date'].dt.week
    df['p_day'] = df['avg purchase date'].dt.day

    
    
    return df

def city_name_encoding(df):
    df_codes = pd.read_csv(os.path.join('data', 'airports.csv'))
    df_codes = df_codes.loc[~df_codes["iata_code"].isnull(), ["iata_code", "municipality"]]
    df_codes.loc[:, "municipality"] = df_codes.loc[:, "municipality"
    ].replace({"Dallas-Fort Worth":"Dallas", "Newark":"New York"})
    df_codes = df_codes.rename(columns={"iata_code":"Departure", "municipality":"Departure City"})
    
    df_encoded = df.merge(df_codes, how="left", on="Departure")
    df_codes = df_codes.rename(columns={"Departure":"Arrival", "Departure City":"Arrival City"})
    df_encoded = df_encoded.merge(df_codes, how="left", on="Arrival")
    return df_encoded

# Code

In [4]:
data1 = pd.read_csv(
    os.path.join('data', 'train.csv.bz2')
)
data2 = pd.read_csv(
    os.path.join('data', 'test.csv.bz2')
) 
data = pd.concat([data1, data2])


In [5]:
print(data["WeeksToDeparture"].mean())

11.466842156704065


In [6]:
airports = ['ORD', 'LAS', 'DEN', 'ATL', 'SFO', 'EWR', 'IAH', 'LAX', 'DFW', 'SEA', 'JFK', 'PHL', 'MIA', 'DTW', 'BOS', 'MSP', 'CLT', 'MCO', 'PHX', 'LGA']

start_date = data['DateOfDeparture'].min()
end_date = data['DateOfDeparture'].max()
all_dates = pd.date_range(start_date, end_date)
dates_df = pd.DataFrame(all_dates, columns=['DateOfDeparture'])
dates_df

df_airports = pd.DataFrame(airports)
df_airports['key'] = 'xyz'
cross_df = pd.merge(df_airports, df_airports,
                    on='key', 
                    suffixes=('_departure', '_arrival')
                    ).drop('key', axis=1)

cross_df['key'] = 'xyz'
dates_df['key'] = 'xyz'

cross_df2 = pd.merge(dates_df, cross_df,
                    on='key'
                    ).drop('key', axis=1)
cross_df2 = cross_df2.rename(columns={'0_departure' : 'Departure', '0_arrival' : 'Arrival'})
cross_df2

,DateOfDeparture,Departure,Arrival
0,2011-09-01,ORD,ORD
1,2011-09-01,ORD,LAS
2,2011-09-01,ORD,DEN
3,2011-09-01,ORD,ATL
4,2011-09-01,ORD,SFO
...,...,...,...
220795,2013-03-05,LGA,MSP
220796,2013-03-05,LGA,CLT
220797,2013-03-05,LGA,MCO
220798,2013-03-05,LGA,PHX


In [8]:
cities = ['Atlanta', 'Boston', 'Chicago', 'Charlotte', 'Dallas', 'Denver', 'Detroit', 'Houston', 'Las Vegas', 'Los Angeles', 'Miami', 'Minneapolis', 'Orlando', 'Philadelphia', 'Phoenix', 'San Francisco', 'New York', 'Seattle', 'Newark']

cities_low = [city.lower() for city in cities]

In [9]:
df = city_name_encoding(cross_df2)
df.head()

# df = df.drop(['log_PAX', 'std_wtd', 'WeeksToDeparture'], axis=1)

,DateOfDeparture,Departure,Arrival,Departure City,Arrival City
0,2011-09-01,ORD,ORD,Chicago,Chicago
1,2011-09-01,ORD,LAS,Chicago,Las Vegas
2,2011-09-01,ORD,DEN,Chicago,Denver
3,2011-09-01,ORD,ATL,Chicago,Atlanta
4,2011-09-01,ORD,SFO,Chicago,San Francisco


In [10]:
print(df.head())

  DateOfDeparture Departure Arrival Departure City   Arrival City
0      2011-09-01       ORD     ORD        Chicago        Chicago
1      2011-09-01       ORD     LAS        Chicago      Las Vegas
2      2011-09-01       ORD     DEN        Chicago         Denver
3      2011-09-01       ORD     ATL        Chicago        Atlanta
4      2011-09-01       ORD     SFO        Chicago  San Francisco


In [11]:
if lag==True:
    df = purchase_date(df)
df['year'] = df['DateOfDeparture'].dt.year
df['month'] = df['DateOfDeparture'].dt.month
df['week'] = df['DateOfDeparture'].dt.week
df['day'] = df['DateOfDeparture'].dt.day

Index(['DateOfDeparture', 'Departure', 'Arrival', 'WeeksToDeparture'], dtype='object')
Index(['DateOfDeparture', 'Departure', 'Arrival', 'Departure City',
       'Arrival City', 'avg purchase date'],
      dtype='object')
  DateOfDeparture Departure Arrival Departure City   Arrival City  \
0      2011-09-01       ORD     ORD        Chicago        Chicago   
1      2011-09-01       ORD     LAS        Chicago      Las Vegas   
2      2011-09-01       ORD     DEN        Chicago         Denver   
3      2011-09-01       ORD     ATL        Chicago        Atlanta   
4      2011-09-01       ORD     SFO        Chicago  San Francisco   
5      2011-09-01       ORD     EWR        Chicago       New York   
6      2011-09-01       ORD     IAH        Chicago        Houston   
7      2011-09-01       ORD     LAX        Chicago    Los Angeles   
8      2011-09-01       ORD     DFW        Chicago         Dallas   
9      2011-09-01       ORD     SEA        Chicago        Seattle   

  avg purchase dat

In [12]:
df
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220800 entries, 0 to 220799
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   DateOfDeparture    220800 non-null  datetime64[ns]
 1   Departure          220800 non-null  object        
 2   Arrival            220800 non-null  object        
 3   Departure City     220800 non-null  object        
 4   Arrival City       220800 non-null  object        
 5   avg purchase date  220800 non-null  object        
 6   p_year             184560 non-null  float64       
 7   p_month            184560 non-null  float64       
 8   p_week             184560 non-null  float64       
 9   p_day              184560 non-null  float64       
 10  year               220800 non-null  int64         
 11  month              220800 non-null  int64         
 12  week               220800 non-null  int64         
 13  day                220800 non-null  int64   

## Airport usual traffic

In [13]:
col_names = ['DateOfDeparture', 'Departure', 'Arrival', 'WeeksToDeparture',
       'log_PAX', 'std_wtd']

# X1, y1 = problem.get_train_data()
# X2, y2 = problem.get_test_data()

# print(X1.shape, X2.shape)

# X_concat = np.concatenate([X1, X2], axis=0)

# print(X_concat[:5])
# y_concat = np.concatenate([y1, y2], axis=0)

# print(X_concat.shape)


# X = pd.DataFrame(data=X_concat, columns= col_names)
# X["Population"] = y_concat

data1 = pd.read_csv(
    os.path.join('data', 'train.csv.bz2')
)
data2 = pd.read_csv(
    os.path.join('data', 'test.csv.bz2')
) 
data = pd.concat([data1, data2])

data = data.drop(columns=["DateOfDeparture", "WeeksToDeparture", "std_wtd"])

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11128 entries, 0 to 2225
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Departure  11128 non-null  object 
 1   Arrival    11128 non-null  object 
 2   log_PAX    11128 non-null  float64
dtypes: float64(1), object(2)
memory usage: 347.8+ KB


In [14]:
data = data.groupby(["Departure", "Arrival"]).mean()
data = data.reset_index()
data = data.rename(columns={"log_PAX" : "Flight baseline frequentation"})

data_merged = df.merge(data, how='left', on=["Departure", "Arrival"])
data_merged["Flight baseline frequentation"] = data_merged["Flight baseline frequentation"].interpolate().fillna(0)
data_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220800 entries, 0 to 220799
Data columns (total 15 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   DateOfDeparture                220800 non-null  datetime64[ns]
 1   Departure                      220800 non-null  object        
 2   Arrival                        220800 non-null  object        
 3   Departure City                 220800 non-null  object        
 4   Arrival City                   220800 non-null  object        
 5   avg purchase date              220800 non-null  object        
 6   p_year                         184560 non-null  float64       
 7   p_month                        184560 non-null  float64       
 8   p_week                         184560 non-null  float64       
 9   p_day                          184560 non-null  float64       
 10  year                           220800 non-null  int64         
 11  

## Weekly google trends ##

In [15]:
df_weekly_google = pd.read_csv('data/weekly_google_searchs.csv')
df_weekly_google = df_weekly_google.drop('Unnamed: 0', axis=1)

In [16]:
df_weekly_google = df_weekly_google.replace({"newark":"New York"})
df_weekly_google['Departure'] = df_weekly_google['Departure'].str.title()
df_weekly_google['Arrival'] = df_weekly_google['Arrival'].str.title()
df_weekly_google = df_weekly_google.rename(columns={"volume": "weekly_volume", "Week": date_key, "Departure": "Departure City","Arrival": "Arrival City" })
df_weekly_google.loc[:, date_key] = pd.to_datetime(df_weekly_google.loc[:, date_key])
df_weekly_google['week'] = df_weekly_google[date_key].dt.week
df_weekly_google['year'] = df_weekly_google[date_key].dt.year
df_weekly_google = df_weekly_google.drop(date_key, axis=1)

In [17]:
df_weekly_google

,Departure City,Arrival City,weekly_volume,week,year
0,Atlanta,Atlanta,40,53,2010
1,Atlanta,Boston,0,53,2010
2,Atlanta,Chicago,0,53,2010
3,Atlanta,Charlotte,32,53,2010
4,Atlanta,Dallas,0,53,2010
...,...,...,...,...,...
74361,New York,Phoenix,13,49,2013
74362,New York,San Francisco,13,49,2013
74363,New York,New York,39,49,2013
74364,New York,Seattle,13,49,2013


In [18]:
data_merged = data_merged.merge(df_weekly_google, how='left', on=['year', 'week', 'Departure City', 'Arrival City'])
#data_merged = data_merged.drop(['week', 'year', 'Departure City', 'Arrival City'], axis=1)

In [19]:
data_merged

,DateOfDeparture,Departure,Arrival,Departure City,Arrival City,avg purchase date,p_year,p_month,p_week,p_day,year,month,week,day,Flight baseline frequentation,weekly_volume
0,2011-09-01,ORD,ORD,Chicago,Chicago,2011-09-01 00:00:00,2011.0,6.0,25.0,20.0,2011,9,35,1,0.000000,49
1,2011-09-01,ORD,LAS,Chicago,Las Vegas,2011-09-01 00:00:00,2011.0,6.0,24.0,15.0,2011,9,35,1,10.618682,0
2,2011-09-01,ORD,DEN,Chicago,Denver,2011-09-01 00:00:00,2011.0,6.0,24.0,14.0,2011,9,35,1,10.787387,19
3,2011-09-01,ORD,ATL,Chicago,Atlanta,2011-09-01 00:00:00,2011.0,6.0,23.0,11.0,2011,9,35,1,11.230863,20
4,2011-09-01,ORD,SFO,Chicago,San Francisco,2011-09-01 00:00:00,2011.0,6.0,24.0,15.0,2011,9,35,1,11.706465,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296235,2013-03-05,LGA,PHX,New York,Phoenix,2013-03-05 00:00:00,2013.0,1.0,4.0,23.0,2013,3,10,5,10.739281,13
296236,2013-03-05,LGA,LGA,New York,New York,2013-03-05 00:00:00,2013.0,1.0,3.0,19.0,2013,3,10,5,10.739281,67
296237,2013-03-05,LGA,LGA,New York,New York,2013-03-05 00:00:00,2013.0,1.0,3.0,19.0,2013,3,10,5,10.739281,0
296238,2013-03-05,LGA,LGA,New York,New York,2013-03-05 00:00:00,2013.0,1.0,3.0,19.0,2013,3,10,5,10.739281,27


In [20]:
data_merged = data_merged.groupby(['DateOfDeparture', 'Departure', 'Arrival', 'Departure City', 'Arrival City']).mean()
data_merged = data_merged.reset_index()

In [21]:
data_merged

,DateOfDeparture,Departure,Arrival,Departure City,Arrival City,p_year,p_month,p_week,p_day,year,month,week,day,Flight baseline frequentation,weekly_volume
0,2011-09-01,ATL,ATL,Atlanta,Atlanta,2011.0,6.0,23.0,11.0,2011.0,9.0,35.0,1.0,10.448246,33.0
1,2011-09-01,ATL,BOS,Atlanta,Boston,2011.0,6.0,25.0,24.0,2011.0,9.0,35.0,1.0,10.842253,0.0
2,2011-09-01,ATL,CLT,Atlanta,Charlotte,NaN,NaN,NaN,NaN,2011.0,9.0,35.0,1.0,10.465763,19.0
3,2011-09-01,ATL,DEN,Atlanta,Denver,2011.0,6.0,24.0,14.0,2011.0,9.0,35.0,1.0,10.402453,19.0
4,2011-09-01,ATL,DFW,Atlanta,Dallas,2011.0,6.0,25.0,20.0,2011.0,9.0,35.0,1.0,11.648248,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220795,2013-03-05,SFO,ORD,San Francisco,Chicago,2013.0,2.0,5.0,1.0,2013.0,3.0,10.0,5.0,11.733258,23.0
220796,2013-03-05,SFO,PHL,San Francisco,Philadelphia,2013.0,1.0,2.0,8.0,2013.0,3.0,10.0,5.0,11.905572,27.0
220797,2013-03-05,SFO,PHX,San Francisco,Phoenix,2012.0,12.0,50.0,10.0,2013.0,3.0,10.0,5.0,11.158633,14.0
220798,2013-03-05,SFO,SEA,San Francisco,Seattle,2012.0,12.0,50.0,10.0,2013.0,3.0,10.0,5.0,11.243991,13.0


## Holidays

In [22]:
df_holidays = pd.read_csv('data/usholidays.csv')
df_holidays

,Unnamed: 0,Date,Holiday
0,0,2010-12-31,New Year's Day
1,1,2011-01-17,"Birthday of Martin Luther King, Jr."
2,2,2011-02-21,Washington's Birthday
3,3,2011-05-30,Memorial Day
4,4,2011-07-04,Independence Day
...,...,...,...
480,480,1966-09-05,Labor Day
481,481,1966-10-12,Columbus Day
482,482,1966-11-11,Veterans Day
483,483,1966-11-24,Thanksgiving Day


In [23]:
df_holidays = df_holidays.drop('Unnamed: 0', axis=1)
df_holidays.loc[:, 'Date'] = pd.to_datetime(df_holidays.loc[:, 'Date'])
df_holidays['year'] = df_holidays['Date'].dt.year
df_holidays['week'] = df_holidays['Date'].dt.week
df_holidays['day'] = df_holidays['Date'].dt.day
df_holidays['Holiday'] = [1 for elem in range(len(df_holidays['Holiday']))]
df_holidays = df_holidays.drop('Date', axis=1)

In [24]:
df_holidays

,Holiday,year,week,day
0,1,2010,52,31
1,1,2011,3,17
2,1,2011,8,21
3,1,2011,22,30
4,1,2011,27,4
...,...,...,...,...
480,1,1966,36,5
481,1,1966,41,12
482,1,1966,45,11
483,1,1966,47,24


In [25]:
data_merged2 = data_merged.merge(df_holidays, how='left', on=['year', 'week', 'day'])
data_merged2 = data_merged2.fillna(0)

In [26]:
data_merged2

,DateOfDeparture,Departure,Arrival,Departure City,Arrival City,p_year,p_month,p_week,p_day,year,month,week,day,Flight baseline frequentation,weekly_volume,Holiday
0,2011-09-01,ATL,ATL,Atlanta,Atlanta,2011.0,6.0,23.0,11.0,2011.0,9.0,35.0,1.0,10.448246,33.0,0.0
1,2011-09-01,ATL,BOS,Atlanta,Boston,2011.0,6.0,25.0,24.0,2011.0,9.0,35.0,1.0,10.842253,0.0,0.0
2,2011-09-01,ATL,CLT,Atlanta,Charlotte,0.0,0.0,0.0,0.0,2011.0,9.0,35.0,1.0,10.465763,19.0,0.0
3,2011-09-01,ATL,DEN,Atlanta,Denver,2011.0,6.0,24.0,14.0,2011.0,9.0,35.0,1.0,10.402453,19.0,0.0
4,2011-09-01,ATL,DFW,Atlanta,Dallas,2011.0,6.0,25.0,20.0,2011.0,9.0,35.0,1.0,11.648248,28.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220795,2013-03-05,SFO,ORD,San Francisco,Chicago,2013.0,2.0,5.0,1.0,2013.0,3.0,10.0,5.0,11.733258,23.0,0.0
220796,2013-03-05,SFO,PHL,San Francisco,Philadelphia,2013.0,1.0,2.0,8.0,2013.0,3.0,10.0,5.0,11.905572,27.0,0.0
220797,2013-03-05,SFO,PHX,San Francisco,Phoenix,2012.0,12.0,50.0,10.0,2013.0,3.0,10.0,5.0,11.158633,14.0,0.0
220798,2013-03-05,SFO,SEA,San Francisco,Seattle,2012.0,12.0,50.0,10.0,2013.0,3.0,10.0,5.0,11.243991,13.0,0.0


## Distances

In [27]:
df_distances = pd.read_csv('data/uscities.csv')
df_distances = df_distances.drop(['Unnamed: 0', 'lat_departure' , 'lng_departure', 'lat_arrival', 'lng_arrival'], axis=1)
df_distances = df_distances.replace({"Dallas-Fort Worth":"Dallas", "Newark":"New York"})
df_distances = df_distances.rename(columns={"city_departure":"Departure City", "city_arrival":"Arrival City"})
df_distances

,Departure City,population_departure,density_departure,Arrival City,population_arrival,density_arrival,distance
0,New York,18713220,10715.0,New York,18713220,10715.0,0.000000
1,New York,18713220,10715.0,Los Angeles,12750807,3276.0,3953.628213
2,New York,18713220,10715.0,Chicago,8604203,4574.0,1155.916177
3,New York,18713220,10715.0,Miami,6445545,5019.0,1756.700669
4,New York,18713220,10715.0,Dallas,5743938,1526.0,2208.283446
...,...,...,...,...,...,...,...
356,New York,282011,4509.0,Denver,2876625,1831.0,2594.877861
357,New York,282011,4509.0,Las Vegas,2104198,1773.0,3578.807900
358,New York,282011,4509.0,Orlando,1822394,1003.0,1510.109592
359,New York,282011,4509.0,Charlotte,1512923,1113.0,846.184556


In [28]:
data_merged3 = data_merged2.merge(df_distances, how='left', on=['Departure City', 'Arrival City'])
data_merged3

,DateOfDeparture,Departure,Arrival,Departure City,Arrival City,p_year,p_month,p_week,p_day,year,...,week,day,Flight baseline frequentation,weekly_volume,Holiday,population_departure,density_departure,population_arrival,density_arrival,distance
0,2011-09-01,ATL,ATL,Atlanta,Atlanta,2011.0,6.0,23.0,11.0,2011.0,...,35.0,1.0,10.448246,33.0,0.0,5449398,1441.0,5449398,1441.0,0.000000
1,2011-09-01,ATL,BOS,Atlanta,Boston,2011.0,6.0,25.0,24.0,2011.0,...,35.0,1.0,10.842253,0.0,0.0,5449398,1441.0,4688346,5532.0,1503.307536
2,2011-09-01,ATL,CLT,Atlanta,Charlotte,0.0,0.0,0.0,0.0,2011.0,...,35.0,1.0,10.465763,19.0,0.0,5449398,1441.0,1512923,1113.0,366.320627
3,2011-09-01,ATL,DEN,Atlanta,Denver,2011.0,6.0,24.0,14.0,2011.0,...,35.0,1.0,10.402453,19.0,0.0,5449398,1441.0,2876625,1831.0,1934.925432
4,2011-09-01,ATL,DFW,Atlanta,Dallas,2011.0,6.0,25.0,20.0,2011.0,...,35.0,1.0,11.648248,28.0,0.0,5449398,1441.0,5743938,1526.0,1151.839803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292003,2013-03-05,SFO,ORD,San Francisco,Chicago,2013.0,2.0,5.0,1.0,2013.0,...,10.0,5.0,11.733258,23.0,0.0,3592294,7256.0,8604203,4574.0,2983.128157
292004,2013-03-05,SFO,PHL,San Francisco,Philadelphia,2013.0,1.0,2.0,8.0,2013.0,...,10.0,5.0,11.905572,27.0,0.0,3592294,7256.0,5649300,4554.0,4054.339226
292005,2013-03-05,SFO,PHX,San Francisco,Phoenix,2012.0,12.0,50.0,10.0,2013.0,...,10.0,5.0,11.158633,14.0,0.0,3592294,7256.0,4219697,1253.0,1043.823889
292006,2013-03-05,SFO,SEA,San Francisco,Seattle,2012.0,12.0,50.0,10.0,2013.0,...,10.0,5.0,11.243991,13.0,0.0,3592294,7256.0,3789215,3469.0,1096.969286


In [29]:
data_merged3 = data_merged3.groupby(['DateOfDeparture', 'Departure', 'Arrival', 'Departure City', 'Arrival City']).mean()
data_merged3 = data_merged3.reset_index()

In [30]:
data_merged3

,DateOfDeparture,Departure,Arrival,Departure City,Arrival City,p_year,p_month,p_week,p_day,year,...,week,day,Flight baseline frequentation,weekly_volume,Holiday,population_departure,density_departure,population_arrival,density_arrival,distance
0,2011-09-01,ATL,ATL,Atlanta,Atlanta,2011.0,6.0,23.0,11.0,2011.0,...,35.0,1.0,10.448246,33.0,0.0,5449398.0,1441.0,5449398.0,1441.0,0.000000
1,2011-09-01,ATL,BOS,Atlanta,Boston,2011.0,6.0,25.0,24.0,2011.0,...,35.0,1.0,10.842253,0.0,0.0,5449398.0,1441.0,4688346.0,5532.0,1503.307536
2,2011-09-01,ATL,CLT,Atlanta,Charlotte,0.0,0.0,0.0,0.0,2011.0,...,35.0,1.0,10.465763,19.0,0.0,5449398.0,1441.0,1512923.0,1113.0,366.320627
3,2011-09-01,ATL,DEN,Atlanta,Denver,2011.0,6.0,24.0,14.0,2011.0,...,35.0,1.0,10.402453,19.0,0.0,5449398.0,1441.0,2876625.0,1831.0,1934.925432
4,2011-09-01,ATL,DFW,Atlanta,Dallas,2011.0,6.0,25.0,20.0,2011.0,...,35.0,1.0,11.648248,28.0,0.0,5449398.0,1441.0,5743938.0,1526.0,1151.839803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220795,2013-03-05,SFO,ORD,San Francisco,Chicago,2013.0,2.0,5.0,1.0,2013.0,...,10.0,5.0,11.733258,23.0,0.0,3592294.0,7256.0,8604203.0,4574.0,2983.128157
220796,2013-03-05,SFO,PHL,San Francisco,Philadelphia,2013.0,1.0,2.0,8.0,2013.0,...,10.0,5.0,11.905572,27.0,0.0,3592294.0,7256.0,5649300.0,4554.0,4054.339226
220797,2013-03-05,SFO,PHX,San Francisco,Phoenix,2012.0,12.0,50.0,10.0,2013.0,...,10.0,5.0,11.158633,14.0,0.0,3592294.0,7256.0,4219697.0,1253.0,1043.823889
220798,2013-03-05,SFO,SEA,San Francisco,Seattle,2012.0,12.0,50.0,10.0,2013.0,...,10.0,5.0,11.243991,13.0,0.0,3592294.0,7256.0,3789215.0,3469.0,1096.969286


## Monthly airports data

In [31]:
data_airports = pd.read_csv('data/airports-passengers-month.csv', sep=';')
data_airports = data_airports.rename(columns={'City_departure' : 'Departure City', 'DOMESTIC' : 'domestic_dep_month_vol', 'INTERNATIONAL' : 'internat_dep_month_vol', 'TOTAL': 'total_dep_month_vol', 'Year':'year', 'Month':'month'})
data_airports

,Departure City,year,month,domestic_dep_month_vol,internat_dep_month_vol,total_dep_month_vol
0,Atlanta,2007,1,2744136,332738,3076874
1,Atlanta,2007,2,2642977,302883,2945860
2,Atlanta,2007,3,3347887,393886,3741773
3,Atlanta,2007,4,3222845,34874,3571585
4,Atlanta,2007,5,3318869,387551,3706420
...,...,...,...,...,...,...
2275,Seattle,2016,8,2057429,240776,2298205
2276,Seattle,2016,9,1679948,209161,1889109
2277,Seattle,2016,10,1586543,190751,1777294
2278,Seattle,2016,11,1510143,170829,1680972


In [32]:
data_merged4 = data_merged3.merge(data_airports, how='left', on=['year', 'month', 'Departure City'])
data_merged4 = data_merged4.drop(['year', 'month'], axis=1)
data_merged4 = data_merged4.drop(['Arrival City', 'Departure City'], axis=1)

In [33]:
data_merged4

,DateOfDeparture,Departure,Arrival,p_year,p_month,p_week,p_day,week,day,Flight baseline frequentation,weekly_volume,Holiday,population_departure,density_departure,population_arrival,density_arrival,distance,domestic_dep_month_vol,internat_dep_month_vol,total_dep_month_vol
0,2011-09-01,ATL,ATL,2011.0,6.0,23.0,11.0,35.0,1.0,10.448246,33.0,0.0,5449398.0,1441.0,5449398.0,1441.0,0.000000,3233004.0,349084.0,3582088.0
1,2011-09-01,ATL,BOS,2011.0,6.0,25.0,24.0,35.0,1.0,10.842253,0.0,0.0,5449398.0,1441.0,4688346.0,5532.0,1503.307536,3233004.0,349084.0,3582088.0
2,2011-09-01,ATL,CLT,0.0,0.0,0.0,0.0,35.0,1.0,10.465763,19.0,0.0,5449398.0,1441.0,1512923.0,1113.0,366.320627,3233004.0,349084.0,3582088.0
3,2011-09-01,ATL,DEN,2011.0,6.0,24.0,14.0,35.0,1.0,10.402453,19.0,0.0,5449398.0,1441.0,2876625.0,1831.0,1934.925432,3233004.0,349084.0,3582088.0
4,2011-09-01,ATL,DFW,2011.0,6.0,25.0,20.0,35.0,1.0,11.648248,28.0,0.0,5449398.0,1441.0,5743938.0,1526.0,1151.839803,3233004.0,349084.0,3582088.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220795,2013-03-05,SFO,ORD,2013.0,2.0,5.0,1.0,10.0,5.0,11.733258,23.0,0.0,3592294.0,7256.0,8604203.0,4574.0,2983.128157,1387834.0,352459.0,1740293.0
220796,2013-03-05,SFO,PHL,2013.0,1.0,2.0,8.0,10.0,5.0,11.905572,27.0,0.0,3592294.0,7256.0,5649300.0,4554.0,4054.339226,1387834.0,352459.0,1740293.0
220797,2013-03-05,SFO,PHX,2012.0,12.0,50.0,10.0,10.0,5.0,11.158633,14.0,0.0,3592294.0,7256.0,4219697.0,1253.0,1043.823889,1387834.0,352459.0,1740293.0
220798,2013-03-05,SFO,SEA,2012.0,12.0,50.0,10.0,10.0,5.0,11.243991,13.0,0.0,3592294.0,7256.0,3789215.0,3469.0,1096.969286,1387834.0,352459.0,1740293.0


In [34]:
#data_merged4 = to_save.merge(data_merged4, how='left', on=['DateOfDeparture', 'Departure', 'Arrival'])

In [35]:
#data_merged4.to_csv('data/multi_sources3.csv', index=False)

## Yahoo - AA

In [36]:
full_df = data_merged4

In [37]:
full_df

,DateOfDeparture,Departure,Arrival,p_year,p_month,p_week,p_day,week,day,Flight baseline frequentation,weekly_volume,Holiday,population_departure,density_departure,population_arrival,density_arrival,distance,domestic_dep_month_vol,internat_dep_month_vol,total_dep_month_vol
0,2011-09-01,ATL,ATL,2011.0,6.0,23.0,11.0,35.0,1.0,10.448246,33.0,0.0,5449398.0,1441.0,5449398.0,1441.0,0.000000,3233004.0,349084.0,3582088.0
1,2011-09-01,ATL,BOS,2011.0,6.0,25.0,24.0,35.0,1.0,10.842253,0.0,0.0,5449398.0,1441.0,4688346.0,5532.0,1503.307536,3233004.0,349084.0,3582088.0
2,2011-09-01,ATL,CLT,0.0,0.0,0.0,0.0,35.0,1.0,10.465763,19.0,0.0,5449398.0,1441.0,1512923.0,1113.0,366.320627,3233004.0,349084.0,3582088.0
3,2011-09-01,ATL,DEN,2011.0,6.0,24.0,14.0,35.0,1.0,10.402453,19.0,0.0,5449398.0,1441.0,2876625.0,1831.0,1934.925432,3233004.0,349084.0,3582088.0
4,2011-09-01,ATL,DFW,2011.0,6.0,25.0,20.0,35.0,1.0,11.648248,28.0,0.0,5449398.0,1441.0,5743938.0,1526.0,1151.839803,3233004.0,349084.0,3582088.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220795,2013-03-05,SFO,ORD,2013.0,2.0,5.0,1.0,10.0,5.0,11.733258,23.0,0.0,3592294.0,7256.0,8604203.0,4574.0,2983.128157,1387834.0,352459.0,1740293.0
220796,2013-03-05,SFO,PHL,2013.0,1.0,2.0,8.0,10.0,5.0,11.905572,27.0,0.0,3592294.0,7256.0,5649300.0,4554.0,4054.339226,1387834.0,352459.0,1740293.0
220797,2013-03-05,SFO,PHX,2012.0,12.0,50.0,10.0,10.0,5.0,11.158633,14.0,0.0,3592294.0,7256.0,4219697.0,1253.0,1043.823889,1387834.0,352459.0,1740293.0
220798,2013-03-05,SFO,SEA,2012.0,12.0,50.0,10.0,10.0,5.0,11.243991,13.0,0.0,3592294.0,7256.0,3789215.0,3469.0,1096.969286,1387834.0,352459.0,1740293.0


In [38]:
start_date = '2011-01-01'
end_date = '2013-12-31'
all_dates = pd.date_range(start_date, end_date)
dates_df = pd.DataFrame(all_dates, columns=[date_key])
dates_df

,Purchase Date
0,2011-01-01
1,2011-01-02
2,2011-01-03
3,2011-01-04
4,2011-01-05
...,...
1091,2013-12-27
1092,2013-12-28
1093,2013-12-29
1094,2013-12-30


In [39]:
data_AA = pd.read_csv('data/yahoo-aairlines.csv')
data_AA = data_AA.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1)
data_AA = data_AA.rename(columns={'Adj Close' : 'a_airlines', 'Date' : date_key})
data_AA.loc[:, date_key] = pd.to_datetime(data_AA.loc[:, date_key])
data_AA = dates_df.merge(data_AA, how='left', on=date_key)
data_AA.loc[:, 'a_airlines'] = data_AA.loc[:, 'a_airlines'].fillna(method='pad')
data_AA.loc[:, 'a_airlines'] = data_AA.loc[:, 'a_airlines'].fillna(method='bfill')
data_AA
### CELUI LA EST OK, A MERGER

,Purchase Date,a_airlines
0,2011-01-01,10.040195
1,2011-01-02,10.040195
2,2011-01-03,10.040195
3,2011-01-04,10.011914
4,2011-01-05,10.464431
...,...,...
1091,2013-12-27,23.511972
1092,2013-12-28,23.511972
1093,2013-12-29,23.511972
1094,2013-12-30,23.361134


In [40]:
data_booking = pd.read_csv('data/yahoo-booking.csv')
data_booking = data_booking.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1)
data_booking = data_booking.rename(columns={'Adj Close' : 'booking', 'Date' : date_key})
data_booking.loc[:, date_key] = pd.to_datetime(data_booking.loc[:, date_key])
data_booking = dates_df.merge(data_booking, how='left', on=date_key)
data_booking.loc[:, 'booking'] = data_booking.loc[:, 'booking'].fillna(method='pad')
data_booking.loc[:, 'booking'] = data_booking.loc[:, 'booking'].fillna(method='bfill')
data_booking
### CELUI LA EST OK, A MERGER

,Purchase Date,booking
0,2011-01-01,415.989990
1,2011-01-02,415.989990
2,2011-01-03,415.989990
3,2011-01-04,409.380005
4,2011-01-05,433.600006
...,...,...
1091,2013-12-27,1170.310059
1092,2013-12-28,1170.310059
1093,2013-12-29,1170.310059
1094,2013-12-30,1152.890015


In [41]:
data_delta = pd.read_csv('data/yahoo-delta.csv')
data_delta = data_delta.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1)
data_delta = data_delta.rename(columns={'Adj Close' : 'delta', 'Date' : date_key})
data_delta.loc[:, date_key] = pd.to_datetime(data_delta.loc[:, date_key])
data_delta = dates_df.merge(data_delta, how='left', on=date_key)
data_delta.loc[:, 'delta'] = data_delta.loc[:, 'delta'].fillna(method='pad')
data_delta.loc[:, 'delta'] = data_delta.loc[:, 'delta'].fillna(method='bfill')
data_delta
### CELUI LA EST OK, A MERGER

,Purchase Date,delta
0,2011-01-01,10.641194
1,2011-01-02,10.641194
2,2011-01-03,10.641194
3,2011-01-04,10.641194
4,2011-01-05,10.641194
...,...,...
1091,2013-12-27,24.132402
1092,2013-12-28,24.132402
1093,2013-12-29,24.132402
1094,2013-12-30,24.275255


In [42]:
full_df = full_df.merge(data_AA, how='left', on=date_key)
full_df = full_df.merge(data_delta, how='left', on=date_key)
full_df = full_df.merge(data_booking, how='left', on=date_key)
full_df

KeyError: 'Purchase Date'

In [42]:
full_df['domestic_dep_month_vol'] = full_df['domestic_dep_month_vol'].fillna(full_df['domestic_dep_month_vol'].mean())
full_df['internat_dep_month_vol'] = full_df['internat_dep_month_vol'].fillna(full_df['internat_dep_month_vol'].mean())
full_df['total_dep_month_vol'] = full_df['total_dep_month_vol'].fillna(full_df['total_dep_month_vol'].mean())
full_df['weekday'] = full_df['DateOfDeparture'].dt.weekday

In [43]:
full_df = full_df.drop(columns=["week", "day"])

if lag:
    full_df = full_df.drop(columns=["Purchase Date", "p_year", "p_month", "p_week", "p_day"])

In [44]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220800 entries, 0 to 220799
Data columns (total 18 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   DateOfDeparture                220800 non-null  datetime64[ns]
 1   Departure                      220800 non-null  object        
 2   Arrival                        220800 non-null  object        
 3   Flight baseline frequentation  220800 non-null  float64       
 4   weekly_volume                  220800 non-null  float64       
 5   Holiday                        220800 non-null  float64       
 6   population_departure           220800 non-null  float64       
 7   density_departure              220800 non-null  float64       
 8   population_arrival             220800 non-null  float64       
 9   density_arrival                220800 non-null  float64       
 10  distance                       220800 non-null  float64       
 11  

In [45]:
full_df.to_csv('data/multi_sources6.csv', index=False)
full_df.to_csv('external_data.csv', index=False)

In [46]:
print(full_df.columns)

Index(['DateOfDeparture', 'Departure', 'Arrival',
       'Flight baseline frequentation', 'weekly_volume', 'Holiday',
       'population_departure', 'density_departure', 'population_arrival',
       'density_arrival', 'distance', 'domestic_dep_month_vol',
       'internat_dep_month_vol', 'total_dep_month_vol', 'a_airlines', 'delta',
       'booking', 'weekday'],
      dtype='object')
